# ASI Hunting - SecurityEvent Template

We would like to guide you though your threat hunting efforts:
    1. Prepare the shell;
    2. Enter values;
    3. Execute queries;
    4. Analyze data.


## 0. Retrieve Log Analytics Information

In [ ]:
from utils import config_reader as mod
workspaceId = mod.config_reader.read_config_values("../config.json")[3]

######################################################################
your_workspace_Name = mod.config_reader.read_config_values("../config.json")[4]
print("Your Log Analytic Workspace: " + your_workspace_Name)
######################################################################

## 1. Install KQL Magic for Log Analytics

In [ ]:
!pip install Kqlmagic --no-cache-dir --upgrade

In [ ]:
%reload_ext Kqlmagic

If you need any help for kql magic     --> %kql --help "help" -popup_window
If you need any help for Log Analytics --> %kql --help "LogAnalytics"

## 2. Log into Azure Notebooks

In [ ]:
print(workspaceId)

In [ ]:
%kql loganalytics://code;workspace=workspaceId

## 3. Collecting parameters

1. Enter How many days looking back

In [ ]:
enteredDaysBack = input()

2. Select an entity type that you want to filter the data

In [ ]:
import ipywidgets as widgets
from IPython.display import display
entityTypes = ['SourceFileName','ClientIP','UserId']
selectedEntityType = widgets.Dropdown(options=entityTypes, value=entityTypes[0],description='Entity Type:')
display(selectedEntityType)

3. Enter an entity value for search

In [ ]:
enteredEntityValue = input()

## 4. KQL operations, results are returned in Pandas DataFrame

### 4.1. SharePoint Download

In [ ]:
%kql OfficeActivity | where RecordType == "SharePointFileOperation" | where Operation in ("FileDownloaded", "FileUploaded") | where TimeGenerated > ago({enteredDaysBack}d) | where {selectedEntityType.value} =~ '{enteredEntityValue}' 
if len(_kql_raw_result_) > 0:
    dfSPDownload = _kql_raw_result_.to_dataframe()
else:
    dfSPDownload = None
    print('no data')

### 4.2  Office 365 Authentications

In [ ]:
%kql OfficeActivity | where RecordType startswith "Az" | where Operation startswith "UserLoggedIn" | where {selectedEntityType.value} =~ '{enteredEntityValue}' | where TimeGenerated > ago({enteredDaysBack}d) | extend UserAgent = extractjson("$[0].Value", ExtendedProperties, typeof(string)) | extend machineIndex=indexof(UserAgent, "machine_id") | extend UserAgent = substring(UserAgent, 0, machineIndex) | project Operation, UserId, TimeGenerated , UserAgent, ClientIP, LogonUserDisplayName, Client_IPAddress, SourceFileName, DestinationFileName 
if len(_kql_raw_result_) > 0:
    dfO365Auth = _kql_raw_result_.to_dataframe()
else:
    dfO365Auth = None
    print('no data')

In [ ]:
dfO365Auth

### 4.3 Office 365 Logons (No Parameters)

In [ ]:
%%kql 
let LogonEvents=() {
let logonFail=OfficeActivity
| where RecordType in ("AzureActiveDirectoryAccountLogon", "AzureActiveDirectoryStsLogon") and ResultStatus =~ "Failed"
| where TimeGenerated > ago(30d) 
| project  EventTime=TimeGenerated, AccountName=split(UserId, "@").[0], AccountDomain = iff(RecordType == "AzureActiveDirectoryAccountLogon",UserDomain,split(UserId, "@").[1]), UserId, IpAddress=ClientIP, OrganizationId, 
ActionType="LogonFailure";
let logonSuccess=OfficeActivity
| where RecordType in ("AzureActiveDirectoryAccountLogon", "AzureActiveDirectoryStsLogon") and ResultStatus =~ "Succeeded"
| where TimeGenerated > ago(30d) 
| project  EventTime=TimeGenerated, AccountName=split(UserId, "@").[0], AccountDomain = iff(RecordType == "AzureActiveDirectoryAccountLogon",UserDomain,split(UserId, "@").[1]), UserId, IpAddress=ClientIP, OrganizationId, 
ActionType="Logon";
 logonFail | union logonSuccess}; 
let logonSummary =
 LogonEvents 
|summarize count() by ActionType, IpAddress, tostring(AccountName), tostring(AccountDomain), UserId, OrganizationId, bin(EventTime, 1m); 
let logon_success = logonSummary | where ActionType == "Logon";
let logon_fail = logonSummary | where ActionType == "LogonFailure";
logon_fail | join kind = leftouter (logon_success) on  IpAddress
| project EventTime, IpAddress, failCount=count_, AccountName, OrganizationId, UserId, successCount=count_1 
| extend successRate = 1.0*successCount/(successCount+failCount)
| project EventTime, IpAddress, AccountName, successRate, failCount, successCount, UserId, OrganizationId

In [ ]:
if len(_kql_raw_result_) > 0:
    dfO365Logons = _kql_raw_result_.to_dataframe()
else:
    dfO365Logons = None
    print('no data')

## 5. Data Analysis

### 5.1 SharePoint Download

In [ ]:
if dfSPDownload is not None:
    dfSPDownloadUserId = dfSPDownload.groupby(['UserId','LogonUserDisplayName'])['UserId'].count()
    dfSPDownloadUserId.plot(kind='bar',x='UserId')

In [ ]:
if dfSPDownload is not None:
    dfSPDownloadClientIP = dfSPDownload.groupby(['ClientIP', 'Client_IPAddress'])['ClientIP'].count()
    dfSPDownloadClientIP.plot(kind='bar',x='ClientIP')

In [ ]:
if dfSPDownload is not None:
    dfSPDownloadFile = dfSPDownload.groupby(['SourceFileName','DestinationFileName'])['DestinationFileName'].count()
    dfSPDownloadFile.plot(kind='bar',x='SourceFileName')

### 5.2 Office 365 Authentications

In [ ]:
if dfO365Auth is not None:
    dfO365AuthUserId = dfO365Auth.groupby(['UserId','LogonUserDisplayName'])['UserId'].count()
    dfO365AuthUserId.plot(kind='bar',x='UserId')

In [ ]:
if dfO365Auth is not None:
    dfO365AuthClientIP = dfO365Auth.groupby(['ClientIP', 'Client_IPAddress'])['ClientIP'].count()
    dfO365AuthClientIP.plot(kind='bar',x='ClientIP')

### 5.3 Office 365 Logons

In [ ]:
display(dfO365Logons)

## 6. Reference Data

In [ ]:
if dfSPDownload is not None:
    dfSPDownload.columns